In [34]:
import math
def Restriksi_Eksponen (a,x,b,c,y,d,e,z,f):
   #Fungsi nilai
   alpha = 1e-2
   try: 
      Result = -math.exp(-alpha/((x-a)*(b-x)))*math.exp(-alpha/((y-c)*(d-y)))*math.exp(-alpha/((z-e)*(f-z)))
   
   except ZeroDivisionError:
      Result = 0.0

   if Result<0:
      IsIn = True
   else:
      IsIn = False

   return IsIn,Result


def Restriksi_Polinom (a,x,b,c,y,d,e,z,f):
   k = 2 #Order
   funciton = -((((x-a)/(b-a))**k)*(1-((x-a)/(b-a)))**k)*((((y-c)/(d-c))**k)*(1-((y-c)/(d-c)))**k)*((((z-e)/(f-e))**k)*(1-((z-e)/(f-e)))**k)
   #Negatif dalam batas
   #0 lainnya
   IsIn=False
   if funciton<0:
      IsIn = True
   else:
      IsIn = False
   return IsIn, funciton
   
   


#Test batas
#Limit X
BackPostX_L = 6.78
BackPostX_U = 6.8

#Limit Y
BackPostY_L = -0.900 #Y>-0.900
BackPostY_U = 0.900 #Y<0.90

#Limit Z
BackPostZ_L = 2.38 #Z>2.38
BackPostZ_U = 3.33 #Z<2.58

X = 6.9
Y = 0.899
Z = 2.38001
X_in = False
Y_in = False
Z_in = False

if BackPostX_L<X<BackPostX_U:
   X_in = True
if BackPostY_L<Y<BackPostY_U:
   Y_in = True
if BackPostZ_L<Z<BackPostZ_U:
   Z_in = True


print(f"Test untuk parameter\n X = ",{X},{X_in},"Batas : ", {BackPostX_L,BackPostX_U})
print(f"Test untuk parameter\n Y = ",{Y},{Y_in},"Batas : ", {BackPostY_L,BackPostY_U})
print(f"Test untuk parameter\n Z = ",{Z},{Z_in},"Batas : ", {BackPostZ_U, BackPostZ_U})

print(f"Hasil Exp",{Restriksi_Eksponen(BackPostX_L,X,BackPostX_U,BackPostY_L,Y,BackPostY_U,BackPostZ_L,Z,BackPostZ_U)})

print(f"Hasil Polinom",{Restriksi_Polinom(BackPostX_L,X,BackPostX_U,BackPostY_L,Y,BackPostY_U,BackPostZ_L,Z,BackPostZ_U)})

Test untuk parameter
 X =  {6.9} {False} Batas :  {6.78, 6.8}
Test untuk parameter
 Y =  {0.899} {True} Batas :  {-0.9, 0.9}
Test untuk parameter
 Z =  {2.38001} {True} Batas :  {3.33}
Hasil Exp {(False, -0.0)}
Hasil Polinom {(True, -3.074386483252624e-14)}


 {(False, -0.0)}


In [95]:
def bismillah (x,y,z,a,b,c,d,e,f):
   SumX = (0.9*(X-a)/(b-a)+0.1)
   SumY = (0.9*(Y-c)/(d-c)+0.1)
   SumZ = (0.9*(Z-e)/(f-e)+0.1)
   return SumX,SumY, SumZ






X,Y,Z = 6.78,0,3.335

Sum = bismillah (X,Y,Z, BackPostX_L,BackPostX_U,BackPostY_L,BackPostY_U,BackPostZ_L,BackPostZ_U)
print(Sum)
print(Sum[0]*Sum[1]*Sum[2])


(0.1, 0.55, 1.0047368421052632)
0.05526052631578948


In [96]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Constants and Parameters (from Appendix A and the paper)
# Basketball properties
circumference = 0.75  # m
mass_ball = 0.600     # kg
radius_ball = circumference / (2 * np.pi)  # ~0.1194 m
cross_area = np.pi * radius_ball**2

# Rim properties
rim_diameter = 0.45   # m
rim_radius = rim_diameter / 2
rim_height = 3.050    # m from floor
rim_coeff_rest = 0.65

# Backboard properties
backboard_width = 1.8  # m
backboard_height = 1.05 # m
backboard_coeff_rest = 0.60

# Court geometry
distance_player_to_hoop = 6.2  # m at 45 degrees
theta_court = np.pi / 4  # 45 degrees in radians

# Backboard position (derived from MATLAB code)
pos_hoop = np.array([
    distance_player_to_hoop * np.cos(theta_court),
    distance_player_to_hoop * np.sin(theta_court),
    rim_height
])
# Backboard position calculation
r_hoop = rim_diameter / 2  # 0.225 m
pos_bb = pos_hoop + np.array([
    0,
    r_hoop + 0.151,  # y-direction offset
    (backboard_height / 2) - 0.15  # z-direction offset
])

# Other parameters
drag_coeff = 0.54
air_density = 1.225  # kg/m³
g = 9.81             # m/s²

# Helper functions
def parallel_perp(v, n):
    """Decompose velocity v into parallel and perpendicular components relative to normal n."""
    n_norm = np.linalg.norm(n)
    if n_norm == 0:
        return np.zeros_like(v), v.copy()
    parallel = np.dot(v, n) / n_norm**2 * n
    perp = v - parallel
    return parallel, perp

def distance_to_rect(w, h, p_ball, p_bb):
    """Calculate distance from ball to backboard rectangle (in y=const plane)."""
    dx = p_ball[0] - p_bb[0]
    dz = p_ball[2] - p_bb[2]
    dy = p_ball[1] - p_bb[1]

    if abs(dx) <= w/2:
        if abs(dz) <= h/2:
            return abs(dy)  # Distance in y-direction
        else:
            dz_clip = h/2 * np.sign(dz)
            edge_z = p_bb[2] + dz_clip
            return np.sqrt(dy**2 + (dz - dz_clip)**2)
    else:
        if abs(dz) <= h/2:
            dx_clip = w/2 * np.sign(dx)
            edge_x = p_bb[0] + dx_clip
            return np.sqrt(dy**2 + (dx - dx_clip)**2)
        else:
            dx_clip = w/2 * np.sign(dx)
            dz_clip = h/2 * np.sign(dz)
            corner = np.array([p_bb[0] + dx_clip, p_bb[1], p_bb[2] + dz_clip])
            return np.linalg.norm(p_ball - corner)

def distance_to_ring(r_rim, p_ball, pos_hoop):
    """Distance from ball to the rim (circle in horizontal plane)."""
    dx = p_ball[0] - pos_hoop[0]
    dy = p_ball[1] - pos_hoop[1]
    horizontal_dist = np.sqrt(dx**2 + dy**2) - r_rim
    vertical_dist = p_ball[2] - pos_hoop[2]
    return np.sqrt(horizontal_dist**2 + vertical_dist**2)

# Event functions
def event_fall_below_rim(t, y):
    return y[2] - (rim_height - 0.01)  # Slightly below rim height
event_fall_below_rim.terminal = True
event_fall_below_rim.direction = -1

def event_collision_backboard(t, y):
    pos = y[:3]
    distance = distance_to_rect(backboard_width, backboard_height, pos, pos_bb)
    return distance - radius_ball
event_collision_backboard.terminal = True
event_collision_backboard.direction = -1

def event_collision_rim(t, y):
    pos = y[:3]
    distance = distance_to_ring(rim_radius, pos, pos_hoop)
    return distance - radius_ball
event_collision_rim.terminal = True
event_collision_rim.direction = -1

# Equations of motion
def equations_of_motion(t, y, omega):
    x, y_pos, z, vx, vy, vz = y
    vel = np.array([vx, vy, vz])
    
    # Gravity
    a_gravity = np.array([0, 0, -g])
    
    # Drag force
    drag_force = -0.5 * drag_coeff * air_density * cross_area * np.linalg.norm(vel) * vel
    a_drag = drag_force / mass_ball
    
    # Magnus force
    magnus_force = (16/3) * np.pi**2 * radius_ball**3 * air_density * np.cross(omega, vel)
    a_magnus = magnus_force / mass_ball
    
    acceleration = a_gravity + a_drag + a_magnus
    return [vx, vy, vz, acceleration[0], acceleration[1], acceleration[2]]

# Main simulation function
def simulate_shot(v0, phi, theta, omega, max_time=10):
    # Initial velocity components
    vx = v0 * np.cos(phi) * np.cos(theta)
    vy = v0 * np.cos(phi) * np.sin(theta)
    vz = v0 * np.sin(phi)
    initial_state = [0, 0, 2, vx, vy, vz]
    
    # Event list
    events = [event_fall_below_rim, event_collision_backboard, event_collision_rim]
    
    # Run simulation
    state = initial_state
    trajectory = []
    times = []
    t_start = 0
    collision_count = 0  # Prevent infinite collisions
    
    while t_start < max_time and collision_count < 5:
        sol = solve_ivp(
            fun=equations_of_motion,
            t_span=(t_start, max_time),
            y0=state,
            args=(omega,),
            events=events,
            rtol=1e-5
        )
        
        trajectory.append(sol.y)
        times.append(sol.t)
        
        if not sol.t_events:
            break
        
        # Check which event triggered
        for i, event_times in enumerate(sol.t_events):
            if len(event_times) > 0:
                last_state = sol.y[:, -1]
                if i == 1:  # Collision with backboard
                    # Compute new velocity
                    velocity = last_state[3:]
                    normal = np.array([0, -1, 0])  # Backboard normal (y-direction)
                    parallel, perp = parallel_perp(velocity, normal)
                    new_velocity = parallel - backboard_coeff_rest * perp
                    state = np.concatenate((last_state[:3], new_velocity))
                    t_start = sol.t[-1]
                    collision_count += 1
                elif i == 2:  # Collision with rim
                    velocity = last_state[3:]
                    # Normal vector at collision point (approximated)
                    pos = last_state[:3]
                    dx = pos[0] - pos_hoop[0]
                    dy = pos[1] - pos_hoop[1]
                    horizontal_dist = np.sqrt(dx**2 + dy**2)
                    normal = np.array([dx, dy, 0]) / horizontal_dist if horizontal_dist != 0 else np.array([0,0,0])
                    parallel, perp = parallel_perp(velocity, normal)
                    new_velocity = parallel - rim_coeff_rest * perp
                    state = np.concatenate((last_state[:3], new_velocity))
                    t_start = sol.t[-1]
                    collision_count += 1
                else:  # Fall below rim
                    break
        else:
            break
    
    # Determine success
    final_pos = np.array([trajectory[-1][0, -1], trajectory[-1][1, -1], trajectory[-1][2, -1]])
    distance = np.linalg.norm(final_pos - pos_hoop)
    success = distance < (rim_radius - radius_ball)
    return success

# Example usage and plotting
if __name__ == "__main__":
    # Example: Varying initial speed and angle
    v0_values = np.linspace(8, 10, 20)
    phi_values = np.linspace(0.7, 1.0, 20)
    success_grid = np.zeros((len(v0_values), len(phi_values)))
    
    for i, v0 in enumerate(v0_values):
        for j, phi in enumerate(phi_values):
            omega = np.array([0, 0, 0])  # No spin
            theta = np.pi / 4  # 45 degrees
            success = simulate_shot(v0, phi, theta, omega)
            success_grid[i, j] = success
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.contourf(v0_values, phi_values, success_grid.T, levels=[0.5, 1.5], colors=['white', 'grey'])
    plt.xlabel('Initial Speed (m/s)')
    plt.ylabel('Elevation Angle (rad)')
    plt.title('Successful Shots (No Spin, θ=π/4)')
    plt.colorbar(ticks=[0, 1], label='Success (1) / Failure (0)')
    plt.show()

TypeError: event_fall_below_rim() takes 2 positional arguments but 3 were given